# PG Databases

Shows DB Size 
Shows DB Activity

TODO: 
- The Notenook uses local methods. Instead, it should use the `DB_Ops` class.

In [10]:
## Read the Connection string from the configu file. 
import configparser

config = configparser.ConfigParser()
config.read_file(open(r'../db_ops.cfg'))

con_str = config.get('con_str', 'PG_TEMBO_DEV1') 

In [12]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from psycopg2 import OperationalError

## Creates a connection to the database using SQLAlchemy.
## Returns:connection: The connection object if successful, None otherwise.
def create_connection(con_str):
    
    try:
        engine = create_engine(con_str)
        connection = engine.connect()
        print("Opened Connection")
        return connection
    except OperationalError as e:
        print("OperationalError: Could not connect to the database. Please check the connection details and ensure the database server is running.")
        print(f"Details: {e}")
    except SQLAlchemyError as e:
        print("SQLAlchemyError: An error occurred while connecting to the database.")
        print(f"Details: {e}")
    except Exception as e:
        print("An unexpected error occurred while connecting to the database.")
        print(f"Details: {e}")
    
    return None

## Execute a query 
def execute_sql_query(con_str, sql_command):
    try:
        connection = create_connection(con_str)
        df = pd.read_sql_query(sql_command, connection)
        return df
    except (SQLAlchemyError, ValueError) as e:
        # Handle any errors or raised exceptions
        raise e
    finally:
        if connection:
            connection.close()

In [11]:
from IPython.display import display

def execute_sql_query(con_str, sql_command):
    try:
        connection = create_connection(con_str)
        df = pd.read_sql_query(sql_command, connection)
        return df
    except (SQLAlchemyError, ValueError) as e:
        raise e
    finally:
        if connection:
            connection.close()

# Define your SQL query
sql_command = """
SELECT 
    oid as db_id, 
    datname as database_name, 
    pg_database_size(datname) / 1024 as database_size_kb,
    pg_size_pretty(pg_database_size(datname)) as db_size
FROM pg_database 
WHERE datistemplate = false;
"""

try:
    # Call the function with both the connection string and SQL query
    df = execute_sql_query(con_str, sql_command)
    
    # Set display options
    pd.set_option('display.max_columns', None)
    pd.set_option('display.expand_frame_repr', False)
    
    # Style the DataFrame
    styled_df = df.style.set_properties(**{'text-align': 'left'}) \
        .set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) \
        .highlight_max(subset=['database_size_kb'], color='red')
    
    # Display the styled DataFrame
    display(styled_df)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")



Opened Connection


,db_id,database_name,database_size_kb,db_size
0,5,postgres,775356,757 MB
1,16385,app,7524,7525 kB
2,19236,metis_ts_dev,217228,212 MB
3,144092,metis,7612,7613 kB
4,16556,demo,797316,779 MB
5,185112,db_10,1375220,1343 MB
6,193461,metis_1,212548,208 MB
7,275674,dvdrental,17884,17 MB
8,299960,plyground,7652,7653 kB
9,398295,pgbench_test,23876,23 MB


In [9]:
import pandas as pd
import plotly.express as px
from IPython.display import display

try:
    # Call the function with both the connection string and SQL query
    df = execute_sql_query(con_str, sql_command)
    
    # Sort the dataframe by size in descending order
    df_sorted = df.sort_values('database_size_kb', ascending=False)
    
    # Get the top 5 databases
    top_5 = df_sorted.head(5)
    
    # Calculate the sum of the rest
    other_sum = df_sorted.iloc[5:]['database_size_kb'].sum()
    
    # Create a new dataframe for the pie chart
    other_df = pd.DataFrame({'database_name': ['ALL_OTHER_DB'], 'database_size_kb': [other_sum]})
    pie_data = pd.concat([top_5, other_df], ignore_index=True)
    
    # Create the pie chart using Plotly Express
    fig = px.pie(pie_data, 
                 values='database_size_kb', 
                 names='database_name', 
                 title='Database Sizes (Top 5 and Others)',
                 hover_data=['database_size_kb'])
    
    # Update the layout for better readability
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    
    # Display the pie chart
    fig.show()
    
    # Display the original dataframe as a styled table
    styled_df = df.style.set_properties(**{'text-align': 'left'}) \
        .set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) \
        .highlight_max(subset=['database_size_kb'], color='red')
    
    display(styled_df)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

Opened Connection


,db_id,database_name,database_size_kb,db_size
0,5,postgres,775356,757 MB
1,16385,app,7524,7525 kB
2,19236,metis_ts_dev,217228,212 MB
3,144092,metis,7612,7613 kB
4,16556,demo,797316,779 MB
5,185112,db_10,1375220,1343 MB
6,193461,metis_1,212548,208 MB
7,275674,dvdrental,17884,17 MB
8,299960,plyground,7652,7653 kB
9,398295,pgbench_test,23876,23 MB
